<a href="https://colab.research.google.com/github/Cepera90/test/blob/main/test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Open inc_points_c
TC = pd.read_csv("/content/drive/MyDrive/test/inc_points_c.csv")
TC = TC.drop(['Unnamed: 0'], axis=1)

In [ ]:
# See median values for drivers and passangers
TC.groupby('TrackOrigin').median()

,Unnamed: 0,IncomingTrackId,Speed,Latitude,Longitude,Height,Acceleration,Deceleration,TickTimestamp,AccelerationXOriginal,AccelerationYOriginal,AccelerationZOriginal
TrackOrigin,,,,,,,,,,,,
OriginalDriver,1383425.0,20681420.0,8.780098,53.214530,45.930267,107091.0,0.0,0.0,1.571309e+09,0.000000,-0.104874,-0.294083
Passanger,3310451.0,20800891.0,31.359722,51.774608,55.232452,110196.0,0.0,0.0,1.571386e+09,-0.002433,0.000000,-0.189388
Taxi,3233483.0,20713602.0,18.775861,55.717843,37.575511,155900.0,0.0,0.0,1.571312e+09,0.000000,0.000000,0.052414


We see what median passanger user speed more when drivers speed (I think because professional drivers have strict speed limit)

In [ ]:
# See mean values for drivers and passangers
TC.groupby('TrackOrigin').mean()

,Unnamed: 0,IncomingTrackId,Speed,Latitude,Longitude,Height,Acceleration,Deceleration,TickTimestamp,AccelerationXOriginal,AccelerationYOriginal,AccelerationZOriginal,MidOverSpeed,HiOverSpeed
TrackOrigin,,,,,,,,,,,,,,
OriginalDriver,1.383425e+06,2.070059e+07,24.503922,34.726617,67.394502,127853.709936,0.045015,0.078361,1.571310e+09,-0.01030,-0.146879,-0.248071,4.768319,0.460334
Passanger,3.303812e+06,2.069142e+07,37.948386,21.225763,84.263927,115249.570732,0.069823,0.182956,1.571297e+09,0.01037,0.028603,-0.154312,10.552048,1.185344
Taxi,3.231599e+06,2.065376e+07,33.141940,51.783496,34.897831,155451.098302,0.264315,0.392949,1.571291e+09,0.03827,-0.044998,0.102176,8.952944,1.044906


We see what mean passanger user speed more when drivers speed. Taxi mean acceleration and deceleration more then OriginalDriver and Taxi.

In [6]:
# Open rich_tracks_2
RT = pd.read_csv("/content/drive/MyDrive/test/rich_tracks_2.csv")

In [13]:
# See standart deviation values for drivers and passangers
RT.groupby('TrackOrigin').std()

,Id,IncomingTrackId,Distance,Duration,AccelerationCount,DecelerationCount,Rating,PhoneUsage,OriginChanged,DistanceGPS,...,CorneringDuration,AccelerationICount,DecelerationICount,AccelerationIDuration,DecelerationIDuration,RatingCornering,VehicleId,BeaconId,DeviationRatio,DeviationMeters
TrackOrigin,,,,,,,,,,,,,,,,,,,,,
OriginalDriver,4801.492524,7.024230e+04,400.357699,28.220975,2.439093,3.154929,1.139604,2.468450,0.0,20.389829,...,1.515978,115.376788,220.519050,184.963118,297.982948,0.701323,NaN,NaN,0.000675,21.208906
Passanger,259711.240231,1.585408e+06,375.529387,38.344564,2.741259,3.029253,1.110365,7.581252,0.0,38.122624,...,9.396443,205.534600,339.757433,561.066446,435.932303,1.097489,NaN,NaN,0.000609,27.464519
Taxi,270605.859722,1.623501e+06,224.925853,29.744323,1.398211,2.161288,1.166092,2.183238,0.0,33.480420,...,1.296748,261.604315,195.238016,540.821350,295.079220,NaN,NaN,NaN,0.000000,26.437740


We see what standart deviation of Duration and PhoneUsage in passanger track data more whan in drivers track data, so we can add duration to inc_point dataframe.

In [14]:
# Create DF with rich_tracks_2 data
ID_t = pd.DataFrame({'IncomingTrackId':RT['IncomingTrackId'],'TrackOrigin':RT['TrackOrigin'],'PhoneUsage':RT['PhoneUsage'],'Duration':RT['Duration']})

In [15]:
# Create DB with track mean data
TrackDB = TC.groupby('IncomingTrackId').mean()
TrackDB = TrackDB.drop(['Latitude','Longitude','Height','TickTimestamp'], axis=1)
TrackDB = TrackDB.rename(columns = {'Speed' : 'Mean_Speed', 'Acceleration' : 'Mean_Acceleration',
                                    'Deceleration' : 'Mean_Deceleration',
                                    'AccelerationXOriginal' : 'Mean_AccelerationX', 
                                    'AccelerationYOriginal' : 'Mean_AccelerationY',
                                    'AccelerationZOriginal' : 'Mean_AccelerationZ'})

In [16]:
TrackDB.head()

,Mean_Speed,Mean_Acceleration,Mean_Deceleration,Mean_AccelerationX,Mean_AccelerationY,Mean_AccelerationZ
IncomingTrackId,,,,,,
19888387,9.988912,0.000000,0.000000,-0.391479,-0.142500,-0.217251
19889292,37.824928,0.000000,0.000000,0.046761,-0.856863,-0.262711
19890131,17.821795,0.000000,0.000000,0.365235,-0.020190,0.101735
19891937,23.292324,0.046806,0.459169,-0.017538,0.174889,0.843765
19892757,10.610393,0.023560,0.231124,0.068676,-0.241727,0.548682


In [18]:
# Merge ID_t with TrackDB
TrackDB = pd.merge(TrackDB, ID_t, on='IncomingTrackId')

In [20]:
TrackDB


,IncomingTrackId,Mean_Speed,Mean_Acceleration,Mean_Deceleration,Mean_AccelerationX,Mean_AccelerationY,Mean_AccelerationZ,TrackOrigin,PhoneUsage,Duration
0,19888387,9.988912,0.000000,0.000000,-0.391479,-0.142500,-0.217251,Passanger,0.000000,1.683333
1,19889292,37.824928,0.000000,0.000000,0.046761,-0.856863,-0.262711,Passanger,0.000000,18.183333
2,19890131,17.821795,0.000000,0.000000,0.365235,-0.020190,0.101735,Passanger,1.166667,24.050000
3,19891937,23.292324,0.046806,0.459169,-0.017538,0.174889,0.843765,Passanger,0.333333,81.783333
4,19892757,10.610393,0.023560,0.231124,0.068676,-0.241727,0.548682,Passanger,0.316667,24.383333
...,...,...,...,...,...,...,...,...,...,...
2582,21497153,73.502072,0.000000,0.000000,-0.038120,0.304370,-0.826014,OriginalDriver,0.000000,44.950000
2583,21499241,7.670491,0.000000,0.000000,0.060988,-0.484137,0.452728,Passanger,0.000000,2.250000
2584,21503950,17.323939,0.012181,0.119491,0.008285,-0.747005,0.297953,Passanger,0.000000,8.066667
2585,21512169,24.370539,0.048752,0.478255,0.267653,0.063431,-0.669810,Passanger,0.550000,16.166667


In [22]:
#Delete IncomingTrackId column
TrackDB = TrackDB.drop(['IncomingTrackId'], axis=1)

In [23]:
# 'OriginalDriver' and 'Taxi' - 'Driver'
TrackDB = TrackDB.replace({'TrackOrigin':{'OriginalDriver':'Driver', 'Taxi':'Driver'}})

In [24]:
TrackDB.loc[:,'TrackOrigin'].unique()

array(['Passanger', 'Driver'], dtype=object)

In [25]:
#Prepare data
X = TrackDB.loc[:,TrackDB.columns != 'TrackOrigin']
y = TrackDB.loc[:,'TrackOrigin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [26]:
# DecisionTreeClassifier model
tree = DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4, splitter="random", random_state=1)

In [27]:
# DecisionTreeClassifier model train
tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4,
                       random_state=1, splitter='random')

In [28]:
# DecisionTreeClassifier model prediction
DTC_pred = tree.predict(X_test)

In [29]:
# Accuracy score DecisionTreeClassifier
print(accuracy_score(DTC_pred, y_test))

0.7818532818532818


Accuracy 0.7818532818532818

In [30]:
# get importance
fi = pd.DataFrame({'feature': list(X.columns),
                   'importance': tree.feature_importances_}).\
                    sort_values('importance', ascending = False)

fi

,feature,importance
0,Mean_Speed,0.295810
4,Mean_AccelerationY,0.162108
7,Duration,0.145628
5,Mean_AccelerationZ,0.140546
3,Mean_AccelerationX,0.092198
1,Mean_Acceleration,0.058901
6,PhoneUsage,0.054877
2,Mean_Deceleration,0.049932


In [31]:
# BaggingClassifier with tree model train
model = BaggingClassifier(DecisionTreeClassifier(splitter='random'), n_estimators=56, random_state=1)
model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(splitter='random'),
                  n_estimators=56, random_state=1)

In [32]:
# BaggingClassifier with tree model prediction
BC_pred = model.predict(X_test)

In [33]:
# Accuracy score BaggingClassifier with tree
print(accuracy_score(BC_pred, y_test))

0.8127413127413128


Accuracy 0.8127413127413128 (n_estimators=56)

In [37]:
# Search parametrs RandomForestClassifier with RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(RandomForestClassifier(), 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train, y_train)
rs.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'bootstrap': False,
 'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 23,
 'n_estimators': 600}

In [38]:
# RandomForestClassifier model prediction
RFCs_pred = rs.predict(X_test)

In [39]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFCs_pred, y_test))

0.8127413127413128


Accuracy 0.8127413127413128

In [47]:
# Search parametrs RandomForestClassifier with GridSearchCV
n_estimators = [400,600,800]
max_features = ['sqrt']
max_depth = [13,15,17]
min_samples_split = [22,23,24]
min_samples_leaf = [2,3,4,5,6,7]
bootstrap = [False]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(RandomForestClassifier(), param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_train, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

Fitting 3 folds for each of 162 candidates, totalling 486 fits


{'bootstrap': False,
 'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 24,
 'n_estimators': 400}

In [49]:
# RandomForestClassifier model prediction
RFCg_pred = gs.predict(X_test)

In [50]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFCg_pred, y_test))

0.8223938223938224


Accuracy 0.8223938223938224

In [41]:
# ExtraTreesClassifier model train
Eforest = ExtraTreesClassifier(n_estimators=600, n_jobs=-1, random_state=1)
Eforest.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=600, n_jobs=-1, random_state=1)

In [42]:
# ExtraTreesClassifier model prediction
EFC_pred = Eforest.predict(X_test)

In [43]:
# Accuracy score ExtraTreesClassifier
print(accuracy_score(EFC_pred, y_test))

0.8185328185328186


Accuracy 0.8185328185328186

In [44]:
# AdaBoostClassifier model train
AB = AdaBoostClassifier(n_estimators=600, random_state=1)
AB.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=600, random_state=1)

In [45]:
# AdaBoostClassifier model prediction
AB_pred = AB.predict(X_test)

In [46]:
# Accuracy score AdaBoostClassifier
print(accuracy_score(AB_pred, y_test))

0.8204633204633205


Accuracy 0.8204633204633205